In [1]:
import pandas as pd

from def_clean import DefensiveCleaning

def_cleaning = DefensiveCleaning(n_cuts=11)

..............................initializing
positional data already downloaded.
reading positional data.
returning positional data.
..data downloaded...


In [2]:
df = def_cleaning.transform_directions(1)

...Week 1 loaded...
...filtered...
...transformed...


In [4]:
df[(df['off'] == False) & (df['position'] == 'TE')]

,time,x,y,s,a,dis,o,dir,event,nflId,...,x_behind_line,x_behind_line_starting,yardline_first_dir,yardline_100_dir,exceeded_1yd,receiver,y_dir,y_dir_qb,y_dir_qb_starting,gamePlayId


In [5]:
def_cleaning.generate_full_df(1, 3)

...Week 1 loaded...
...filtered...
...transformed...
...time reduced...
...starting dataframe generated...
...action type generated...
.....Week 1 COMPLETE.....

   33.33% COMPLETE   

--- 0.5 minutes elapsed ---

the weeks complete:  [1]
...Week 2 loaded...
...filtered...
...transformed...
...time reduced...
...starting dataframe generated...
...action type generated...
.....Week 2 COMPLETE.....

   66.67% COMPLETE   

--- 1.1 minutes elapsed ---

the weeks complete:  [1 2]
...Week 3 loaded...
...filtered...
...transformed...
...time reduced...
...starting dataframe generated...
...action type generated...
.....Week 3 COMPLETE.....

   100.0% COMPLETE   

--- 1.6 minutes elapsed ---

the weeks complete:  [1 2 3]
Defensive cleaning complete --- check assets/def_clean_output.csv


posId                                                                                        CBL0_act  \
gameId     playId defendersInTheBox numberOfPassRushers DB LB DL yardline_first yardline_100            
2018090600 75     7.0               4.0                 5  2  4  65.0           80.0                M   
           146    7.0               4.0                 5  2  4  29.0           39.0                M   
           168    6.0               4.0                 5  2  4  29.0           39.0                Z   
           190    6.0               5.0                 6  1  4  29.0           39.0                Z   
           256    8.0               6.0                 2  3  6  0.0            1.0               NaN   
...                                                                                               ...   
2018092400 4029   6.0               4.0                 5  2  4  73.0           64.0                M   
           4121   6.0               4.0                 5  4  2  70.0           80.0                Z   
           4165   5.0               5.0                 6  3  2  70.0           80.0                M   
           4199   6.0               4.0                 6  3  2  70.0           80.0                M   
           4286   7.0               5.0                 4  3  4  42.0           32.0                M   

posId                                                                                        CBL0_x_start  \
gameId     playId defendersInTheBox numberOfPassRushers DB LB DL yardline_first yardline_100                
2018090600 75     7.0               4.0                 5  2  4  65.0           80.0                 7.37   
           146    7.0               4.0                 5  2  4  29.0           39.0                 2.72   
           168    6.0               4.0                 5  2  4  29.0           39.0                 1.62   
           190    6.0               5.0                 6  1  4  29.0           39.0                 1.62   
           256    8.0               6.0                 2  3  6  0.0            1.0                   NaN   
...                                                                                                   ...   
2018092400 4029   6.0               4.0                 5  2  4  73.0           64.0                 0.98   
           4121   6.0               4.0                 5  4  2  70.0           80.0                 0.76   
           4165   5.0               5.0                 6  3  2  70.0           80.0                 4.25   
           4199   6.0               4.0                 6  3  2  70.0           80.0                 0.38   
           4286   7.0               5.0                 4  3  4  42.0           32.0                 0.88   

posId                                                                                        CBL0_y_start  \
gameId     playId defendersInTheBox numberOfPassRushers DB LB DL yardline_first yardline_100                
2018090600 75     7.0               4.0                 5  2  4  65.0           80.0                 6.13   
           146    7.0               4.0                 5  2  4  29.0           39.0                 0.59   
           168    6.0               4.0                 5  2  4  29.0           39.0                 5.45   
           190    6.0               5.0                 6  1  4  29.0           39.0                 4.85   
           256    8.0               6.0                 2  3  6  0.0            1.0                   NaN   
...                                                                                                   ...   
2018092400 4029   6.0               4.0                 5  2  4  73.0           64.0                19.99   
           4121   6.0               4.0                 5  4  2  70.0           80.0                 2.06   
           4165   5.0               5.0                 6  3  2  70.0           80.0                 4.94   
           4199  